## Sub Holding Keys

By default, LUSID separates holdings by instrument. Sub-holding keys are labels which allow a more flexible grouping of holdings within instrument groups - this is useful for e.g. bucketing cash held in a single currency, since it allows one to track cash flows and logically partition holdings into sub-holdings.

Portfolios can be initialised with a sub_holding_keys array, and transactions can be painted for bucketing with properties that contain matching keys.

See more in the knowledge base: https://support.lusid.com/what-are-subholding-keys

This use-case will demonstrate:

1. The default behaviour of LUSID upon taking holdings
2. The initialisation of a new portfolio with sub-holding keys
3. The upsertion of properties to existing transactions
4. The results of bucketing via sub-holding keys

# 0. Initial Setup

In [1]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
# Import Libraries
import pprint
from datetime import datetime, timedelta, time, date
import pytz
import uuid
import printer as prettyprint
from datetime import datetime
import pandas as pd
import numpy as np
import os
import json

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print('LUSID Environment Initialised')
print('LUSID version : ', client.metadata.get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID version :  0.5.3139.0


# 1. LUSID Without Sub-holding Keys

Define the parameters of a new portfolio, request it and create the portfolio in LUSID:

In [2]:
# The following function creates a random alphanumeric code of 4 characters that can be appended to Ids
# and Names to ensure they remain unique throughout multiple runs of this example
def get_guid():
    return str(uuid.uuid4())[:12]

guid = get_guid()

portfolio_code = 'SHKey_Demo'+"-"+guid
portfolio_display_name = "portfolio-{0}".format(guid)

scope = "use-case-SHK"

# Set the creation date of the portfolio and today's date
portfolio_date = datetime(2015, 1, 1, tzinfo=pytz.utc)
today_date = date.today()

In [3]:
# Set up the portfolio: CREATE a request
request = models.CreateTransactionPortfolioRequest(
    
    # descriptive name for the portfolio
    display_name=portfolio_display_name,

    # unique portfolio code, portfolio codes must be unique across scopes
    code=portfolio_code,
    base_currency="GBP",
    created = portfolio_date)

# and THEN submit response to lusid
result = client.transaction_portfolios.create_portfolio(
    scope = scope,
    create_request=request)


Begin our investment campaign with an initial sum of £2000:

In [4]:
# Upsert an initial £2000 investment

initial_fund_response = models.TransactionRequest(
        # unique transaction id
        transaction_id=get_guid(),

        # transaction type, configured during system setup
        type="FundsIn",

        # Cash instruments are identified using CCY_ followed by the ISO currency codes.
        # Cash instruments do not need to be created before use
        instrument_identifiers={"Instrument/default/Currency": "GBP"},

        transaction_date=portfolio_date,
        settlement_date=portfolio_date,
        transaction_price=models.TransactionPrice(1),
        units=2000,
        total_consideration=models.CurrencyAndAmount(2000, "GBP"),
        source="Client")

client.transaction_portfolios.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transactions=[initial_fund_response]);

## Add Transactions

Suppose we hold some equity and some bonds. If the equity pays dividends and the bonds pay a coupon, then there are two separate cash flows into the portfolio from each of these sources. We'll represent purchase of the instruments and their corresponding cash incomes as transactions.

Transactions for the purchase of instruments:

In [5]:
transactions = []

# A transaction to buy 100 units of Figi BBG000BKH1W6 (Kingfisher_LondonStockEx_KGF, pays dividends)

KF_unit_price = 2.276
KF_units = 100
KF_total_price = KF_unit_price * KF_units

# Date of equity purchase
KF_date = datetime(2015, 9, 7, tzinfo=pytz.utc)

Equity = models.TransactionRequest(

    # unique transaction id
    transaction_id=get_guid(),

    # transaction type, configured during system setup
    type="Buy",
    instrument_identifiers={
        'Instrument/default/Figi': "BBG000BKH1W6"
    },
    transaction_date=KF_date,
    settlement_date=KF_date,
    units=KF_units,
    transaction_price=models.TransactionPrice(price=KF_unit_price),
    total_consideration=models.CurrencyAndAmount(amount=KF_total_price, currency="GBP"),
    source="Client"
    )

transactions.append(Equity)

# A second transaction to buy 10 units of Figi BBG0088JSC32 (UK GILT STOCK 2% 2025 Maturity)

GILT_unit_price = 106.637
GILT_units = 10
GILT_total_price = GILT_unit_price*GILT_units

# Date of bond purchase
GILT_date = datetime(2015, 5, 13, tzinfo=pytz.utc)

Bond = models.TransactionRequest(

    # unique transaction id
    transaction_id=get_guid(),

    # transaction type, configured during system setup
    type="Buy",
    instrument_identifiers={
        'Instrument/default/Figi': "BBG0088JSC32"
    },
    transaction_date=GILT_date,
    settlement_date=GILT_date,
    units=GILT_units,
    transaction_price=models.TransactionPrice(price=GILT_unit_price),
    total_consideration=models.CurrencyAndAmount(amount=GILT_total_price, currency="GBP"),
    source="Client"
    )

transactions.append(Bond)

Transactions for cash incomes (coupon and dividend payments):

In [6]:
# Transactions for the dividend cash flow

# Dates on which dividends were payed
div_dates = [datetime(2015,11,13, tzinfo=pytz.utc), datetime(2016,6,20, tzinfo=pytz.utc), datetime(2016,11,11, tzinfo=pytz.utc),
              datetime(2017,6,19, tzinfo=pytz.utc), datetime(2017,11,10, tzinfo=pytz.utc), datetime(2018,6,18, tzinfo=pytz.utc),
              datetime(2018,11,9, tzinfo=pytz.utc), datetime(2019,7,15, tzinfo=pytz.utc)]
# Corresponding Rates
dividend_rates = [3.18,6.92,3.25,7.15,3.33,7.49,3.33,7.49]

for i in range(len(div_dates)):
    transactions.append(models.TransactionRequest(
            # unique transaction id
            transaction_id="dividend"+get_guid(),

            # transaction type, configured during system setup
            type="FundsIn",

            # Cash instruments are identified using CCY_ followed by the ISO currency codes.
            # Cash instruments do not need to be created before use
            instrument_identifiers={"Instrument/default/Currency": "GBP"},

            transaction_date=div_dates[i],
            settlement_date=div_dates[i],
            transaction_price=models.TransactionPrice(1),
            units=KF_units*dividend_rates[i]/100,
            total_consideration=models.CurrencyAndAmount(KF_total_price*dividend_rates[i]/100, "GBP"),
            source="Client")
        )
    
# Transactions for the coupon cash flow

# Dates on which coupon payments were recieved
coupon_dates = [datetime(2015,7,9, tzinfo=pytz.utc), datetime(2016,7,3, tzinfo=pytz.utc), datetime(2016,7,9, tzinfo=pytz.utc), 
               datetime(2017,7,3, tzinfo=pytz.utc), datetime(2017,7,9, tzinfo=pytz.utc), datetime(2018,7,3, tzinfo=pytz.utc), 
               datetime(2018,7,9, tzinfo=pytz.utc), datetime(2019,7,3, tzinfo=pytz.utc)]

coupon_value = 2*GILT_units

for i in range(len(coupon_dates)):
    transactions.append(models.TransactionRequest(
            # unique transaction id
            transaction_id="coupon"+get_guid(),

            # transaction type, configured during system setup
            type="FundsIn",

            # Cash instruments are identified using CCY_ followed by the ISO currency codes.
            # Cash instruments do not need to be created before use
            instrument_identifiers={"Instrument/default/Currency": "GBP"},

            transaction_date=coupon_dates[i],
            settlement_date=coupon_dates[i],
            transaction_price=models.TransactionPrice(1),
            units=coupon_value,
            total_consideration=models.CurrencyAndAmount(coupon_value, "GBP"),
            source="Client")
        )

Upsert transactions and check response:

In [7]:
# Upsert Transactions to portfolio

response = client.transaction_portfolios.upsert_transactions(
    scope=scope,
    code=portfolio_code,
    transactions=transactions)

#Print the response from LUSID using pretty formatting 
prettyprint.transactions_response(
    response,
    scope,
    portfolio_code)

Transactions Successfully Upserted into Portfolio
Scope: use-case-SHK
Code: SHKey_Demo-9d1a1f48-568
Transactions Effective From: 2019-07-15 00:00:00+00:00
Transactions Created On: 2019-08-23 13:35:41.597285+00:00



Take holdings: This cell demonstrates that the default behaviour of lusid is to separate holdings at the instrument level, i.e. that all of our cash (from the initial investment and both coupon/dividend incomes) is grouped together.

In [8]:
holdings_response = client.transaction_portfolios.get_holdings(scope=scope, 
                                                               code=portfolio_code,
                                                               effective_at=today_date)

prettyprint.holdings_response_instrument_id(holdings_response)

,Units,Cost,Currency,Instrument Id
0,10.00,1066.37,GBP,LUID_ZTMR6A5H
1,100.00,227.60,GBP,LUID_H6LL1EIM
2,908.17,908.17,GBP,CCY_GBP


# 2. Portfolios With Sub-Holding Keys

LUSID supports a more flexible bucketing of instruments into holdings. *Sub-Holding Keys* are labels which can be used to group instruments within a holding into separate buckets: sub-holdings.

Portfolios must be initialised with a list of subholding keys, and these cannot be changed later. This is because they are central to the way LUSID responds to a get_holdings request.

To demonstrate this feature, we will create a copy of our original portfolio (a derived transaction portfolio) and initialise it with subholding keys.

In [9]:
# The keys that label the cash buckets
sub_holding_keys = ["Transaction/"+scope+"/coupon", "Transaction/"+scope+"/dividend", "Transaction/"+scope+"/fund"]

derived_portfolio_code = "derived-"+portfolio_code

derived_portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
    # this uses the portfolio name
    display_name="derived-"+portfolio_display_name,
    code=derived_portfolio_code,
    parent_portfolio_id=models.ResourceId(
        scope=scope,
        code=portfolio_code),
    description="portfolio with sub-holding keys",
    created=portfolio_date,
    corporate_action_source_id=None,
    accounting_method=None,
    # Initialise with sub-holding keys
    sub_holding_keys=sub_holding_keys)

derived_portfolio = client.derived_transaction_portfolios.create_derived_portfolio(
    scope=scope,
    portfolio=derived_portfolio_request)

prettyprint.portfolio_response(derived_portfolio)

Derived Portfolio Created
Scope: use-case-SHK
Code: derived-SHKey_Demo-9d1a1f48-568
Portfolio Effective From: 2015-01-01 00:00:00+00:00
Portfolio Created On: 2019-08-23 13:35:42.036232+00:00

   Parent Portfolio Details
   Scope: use-case-SHK
   Code: SHKey_Demo-9d1a1f48-568



It's also possible to specify sub-holding keys for transaction portfolios that don't derive from other portfolios: this is done simply by initialising the portfolio request and passing an array to the sub_holding_keys optional argument, just like in the derived portfolio request above.

# 3. Transaction Properties

## 3.1 Upsert Properties to Existing Transactions
Create Properties to attach to coupon and dividend transactions:

In [10]:
dividend_property_request = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=scope,
    code='dividend',
    value_required=True,
    display_name='dividend_cash',
    life_time='Perpetual',
    data_type_id=models.ResourceId(
        scope='system', 
        code='string'))

coupon_property_request = models.CreatePropertyDefinitionRequest(
    domain='Transaction',
    scope=scope,
    code='coupon',
    value_required=True,
    display_name='equity_cash',
    life_time='Perpetual',
    data_type_id=models.ResourceId(
        scope='system', 
        code='string'))

In [11]:
# Create property definitions from requests
dividend_property_response = client.property_definitions.create_property_definition(
    definition=dividend_property_request)

coupon_property_response = client.property_definitions.create_property_definition(
    definition=coupon_property_request)

prettyprint.property_response(dividend_property_response)
prettyprint.property_response(coupon_property_response)

Property Response Details
Scope: use-case-SHK
Code: dividend
Lifetime: Perpetual

Property Response Details
Scope: use-case-SHK
Code: coupon
Lifetime: Perpetual



Now that we have the property definitions (the property_responses elements) we can actually build the properties:

In [12]:
dividend_property =  models.PerpetualProperty(key=dividend_property_response.key,
                                              value=models.PropertyValue(
                                                  label_value=dividend_property_response.code))

coupon_property =  models.PerpetualProperty(key=coupon_property_response.key,
                                              value=models.PropertyValue(
                                                  label_value=coupon_property_response.code))

Upsert transaction properties:

In [13]:
# LUSID doesn't directly support property upsertion to inherited transactions in a derived portfolio. Upsert properties to the
# transactions in the parent portolio; the inherited transactions will be automatically updated with these changes.

# Loop through transactions, check to see if they should be categorised as coupon or dividend.
# If so, categorise by attaching a property.
for each_transaction in transactions:
    
    if each_transaction.transaction_id.startswith("dividend"):
        client.transaction_portfolios.upsert_transaction_properties(scope=scope,
                                           code=portfolio_code,
                                           transaction_id=each_transaction.transaction_id,
                                           transaction_properties={dividend_property_response.key:dividend_property} )
        
    if each_transaction.transaction_id.startswith("coupon"):
        client.transaction_portfolios.upsert_transaction_properties(scope=scope,
                                           code=portfolio_code,
                                           transaction_id=each_transaction.transaction_id,
                                           transaction_properties={coupon_property_response.key:coupon_property} )

Properties can also be attached to transactions upon initialisation (as optional arguments of the request call). As mentioned, non-derived portfolios can be initialised with sub-holding keys, so it is possible to use the functionality provided by LUSID's sub-holding keys from the outset (rather than via retrospect.)

# 4. Take Holdings

Now when we take holdings, notice that there are three cash buckets: coupons, dividends, and uncategorised (leftover cash.)

In [14]:
new_holdings_response = client.transaction_portfolios.get_holdings(scope=scope, 
                                                               code=derived_portfolio_code,
                                                               effective_at=today_date)

prettyprint.holdings_response_instrument_id_shk(response=new_holdings_response)

,Units,Cost,Currency,Instrument Id,Sub-holding Key
0,10.00,1066.37,GBP,LUID_ZTMR6A5H,none
1,100.00,227.60,GBP,LUID_H6LL1EIM,none
2,706.03,706.03,GBP,CCY_GBP,none
3,160.00,160.00,GBP,CCY_GBP,coupon
4,42.14,42.14,GBP,CCY_GBP,dividend


Transactions can possess properties that match multiple sub-holding keys, meaning that they may be grouped into more than one sub-holding. This is how LUSID can provide more flexible accounting capabilities.

Tidy up; delete property definitions

In [15]:
client.property_definitions.delete_property_definition(domain="Transaction",scope=scope,code='dividend');
client.property_definitions.delete_property_definition(domain="Transaction",scope=scope,code='coupon');